In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [92]:
%pip install nltk
import nltk
%pip install langdetect
from langdetect import detect

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [93]:
lyrics = pd.read_csv("songs_lyrics.csv", sep="|", na_values={"": None}, keep_default_na=False)
lyrics

,title,artist,lyrics
0,Flowers,Miley Cyrus,88 ContributorsTranslationsEspañolPortuguêsKis...
1,Cupid - Twin Ver.,FIFTY FIFTY,1 ContributorTranslationsEnglishBahasa MelayuF...
2,BESO,ROSALÍA,29 ContributorsTranslationsPortuguêsEnglishDeu...
3,Boy's a liar Pt. 2,PinkPantheress,92 ContributorsTranslationsPortuguêsTürkçeDeut...
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2 ContributorsNew Music Friday 12/02/22 Lyrics...
...,...,...,...
2380,Southbound,Beach Blvd,2 ContributorsMe & My Skate Lyrics[Skit]\nOh s...
2381,Dance with Somebody - Radio Version,Mando Diao,1 ContributorDance With Somebody - Radio Versi...
2382,Flow,Desire Machines,1 ContributorWhat Matters? Lyrics[Hook]\nLeave...
2383,Scared of the Dark,Everything Brighter,1 ContributorWay Home LyricsYa Ya Ya\n\nI hear...


In [94]:
songs = pd.read_csv("songs.csv", sep=";")
popularity = songs["popularity"]
lyrics["popularity"] = popularity
lyrics

,title,artist,lyrics,popularity
0,Flowers,Miley Cyrus,88 ContributorsTranslationsEspañolPortuguêsKis...,98
1,Cupid - Twin Ver.,FIFTY FIFTY,1 ContributorTranslationsEnglishBahasa MelayuF...,97
2,BESO,ROSALÍA,29 ContributorsTranslationsPortuguêsEnglishDeu...,96
3,Boy's a liar Pt. 2,PinkPantheress,92 ContributorsTranslationsPortuguêsTürkçeDeut...,96
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,2 ContributorsNew Music Friday 12/02/22 Lyrics...,96
...,...,...,...,...
2380,Southbound,Beach Blvd,2 ContributorsMe & My Skate Lyrics[Skit]\nOh s...,21
2381,Dance with Somebody - Radio Version,Mando Diao,1 ContributorDance With Somebody - Radio Versi...,16
2382,Flow,Desire Machines,1 ContributorWhat Matters? Lyrics[Hook]\nLeave...,15
2383,Scared of the Dark,Everything Brighter,1 ContributorWay Home LyricsYa Ya Ya\n\nI hear...,11


In [95]:
print(f"Ci sono {(lyrics['lyrics'] == '').sum()} canzoni di cui non è stato trovato il testo.")
lyrics[lyrics["lyrics"] == ''].head()

Ci sono 54 canzoni di cui non è stato trovato il testo.


,title,artist,lyrics,popularity
16,Until I Found You (with Em Beihold) - Em Beiho...,Stephen Sanchez,,93
317,"Happy - From ""Despicable Me 2""",Pharrell Williams,,82
339,"Moves Like Jagger - Studio Recording From ""The...",Maroon 5,,82
446,CAN'T STOP THE FEELING! (from DreamWorks Anima...,Justin Timberlake,,80
684,Valerie (feat. Amy Winehouse) - Version Revisited,Mark Ronson,,78


In [96]:
lyrics["lyrics"][0]

"88 ContributorsTranslationsEspañolPortuguêsKiswahiliDeutschItalianoСрпскиNederlandsSvenskaPolskiTürkçeΕλληνικάFrançaisDanskРусскийالعربيةУкраїнськаBahasa IndonesiaFlowers Lyrics[Verse 1]\nWe were good, we were gold\nKinda dream that can't be sold\nWe were right 'til we weren't\nBuilt a home and watched it burn\n\n[Pre-Chorus]\nMm, I didn't wanna leave you, I didn't wanna lie\nStarted to cry, but then remembered I\n\n[Chorus]\nI can buy myself flowers\nWrite my name in the sand\nTalk to myself for hours\nSay things you don't understand\nI can take myself dancing\nAnd I can hold my own hand\nYeah, I can love me better than you can\n\n[Post-Chorus]\nCan love me better, I can love me better, baby\nCan love me better, I can love me better, baby\n\n[Verse 2]\nPaint my nails cherry-red\nMatch the roses that you left\nNo remorse, no regret\nI forgive every word you said\nYou might also like[Pre-Chorus]\nOoh, I didn't wanna leave you, baby, I didn't wanna fight\nStarted to cry, but then rememb

In [97]:
def clean_lyrics(text):
    if text != '':
        contributors, other = text.split("Contributor")
        text = other.split("Lyrics")[1].split("Embed")[0] if int(contributors) > 20 else ''
    return text

In [98]:
lyrics["lyrics"] = lyrics["lyrics"].map(clean_lyrics)
print(f"Ci sono {(lyrics['lyrics'] == '').sum()} canzoni che ora non hanno il testo.")
lyrics

Ci sono 775 canzoni che ora non hanno il testo.


,title,artist,lyrics,popularity
0,Flowers,Miley Cyrus,"[Verse 1]\nWe were good, we were gold\nKinda d...",98
1,Cupid - Twin Ver.,FIFTY FIFTY,,97
2,BESO,ROSALÍA,"[Letra de ""BESO""]\n\n[Coro: ROSALÍA]\nYa yo ne...",96
3,Boy's a liar Pt. 2,PinkPantheress,[Verse 1: PinkPantheress]\nTake a look inside ...,96
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,,96
...,...,...,...,...
2380,Southbound,Beach Blvd,,21
2381,Dance with Somebody - Radio Version,Mando Diao,,16
2382,Flow,Desire Machines,,15
2383,Scared of the Dark,Everything Brighter,,11


In [99]:
lyrics[lyrics["lyrics"] != '']

,title,artist,lyrics,popularity
0,Flowers,Miley Cyrus,"[Verse 1]\nWe were good, we were gold\nKinda d...",98
2,BESO,ROSALÍA,"[Letra de ""BESO""]\n\n[Coro: ROSALÍA]\nYa yo ne...",96
3,Boy's a liar Pt. 2,PinkPantheress,[Verse 1: PinkPantheress]\nTake a look inside ...,96
5,Daylight,David Kushner,[Verse 1]\nTelling myself I won't go there\nOh...,96
6,TQG,KAROL G,"[Letra de ""TQG""]\n\n[Intro: KAROL G]\nLa que t...",96
...,...,...,...,...
2349,Lips Of An Angel,Hinder,"[Verse 1]\nHoney, why you callin’ me so late?\...",38
2351,Fake It,Seether,[Verse 1]\nWho's to know if your soul will fad...,37
2355,Switch,Will Smith,"[Intro]\nYo mic check, mic check, yeah here yo...",36
2361,The Selfish Gene,Marlon Percy,"5/1\n11:11 - ""Me, Chanel & Jameson""\nAazar & Y...",33


In [100]:
def detect_language(text):
    return detect(text) if text != '' else None

In [101]:
lyrics["language"] = lyrics["lyrics"].apply(detect_language)
lyrics

,title,artist,lyrics,popularity,language
0,Flowers,Miley Cyrus,"[Verse 1]\nWe were good, we were gold\nKinda d...",98,en
1,Cupid - Twin Ver.,FIFTY FIFTY,,97,None
2,BESO,ROSALÍA,"[Letra de ""BESO""]\n\n[Coro: ROSALÍA]\nYa yo ne...",96,es
3,Boy's a liar Pt. 2,PinkPantheress,[Verse 1: PinkPantheress]\nTake a look inside ...,96,en
4,Creepin' (with The Weeknd & 21 Savage),Metro Boomin,,96,None
...,...,...,...,...,...
2380,Southbound,Beach Blvd,,21,None
2381,Dance with Somebody - Radio Version,Mando Diao,,16,None
2382,Flow,Desire Machines,,15,None
2383,Scared of the Dark,Everything Brighter,,11,None


In [102]:
lyrics["language"].value_counts()

en    1559
es      39
id       4
fr       2
lv       1
cy       1
ro       1
tl       1
pt       1
so       1
Name: language, dtype: int64

In [103]:
#clean_dataset = lambda df: df[df["lyrics"] != '']
#lyrics = clean_dataset(lyrics)

In [111]:
from sklearn.feature_extraction.text import CountVectorizer

cvect = CountVectorizer(min_df=15, stop_words="english")
mat = cvect.fit_transform(lyrics["lyrics"])
lyric_popularity = mat.toarray().sum(axis=1)
np.corrcoef(lyric_popularity, lyrics["popularity"])

array([[1.        , 0.16869965],
       [0.16869965, 1.        ]])

In [148]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

def print_eval(model, X, y_true):
    y_pred = model.predict(X_val)
    print("R2:", r2_score(y_true=y_val, y_pred=y_pred))
    print("RMSE:", np.sqrt(mean_squared_error(y_true=y_val, y_pred=y_pred)))
    print("MAE:", mean_absolute_error(y_true=y_val, y_pred=y_pred))
    print("MAPE:", mean_absolute_percentage_error(y_true=y_val, y_pred=y_pred))

In [149]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(lyrics["lyrics"], lyrics["popularity"], test_size=1/3, random_state=42)

In [155]:
def create_features_dict(model):
    features = model["vect"].get_feature_names_out()
    print("Valid words:", len(features))
    coefs = model["regr"].coef_
    features_coefs = {features[i]: coefs[i] for i in range(len(features)) if abs(coefs[i]) > 0.01}
    print("Valuable words:", len(features_coefs))
    return features_coefs

In [171]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold, GridSearchCV

model = Pipeline([
    ("vect", CountVectorizer(min_df=0.05, stop_words="english")),
    ("regr", Lasso(alpha=5))
])

model.fit(X_train, y_train)
print_eval(model, X_val, y_val)
features_coefs = create_features_dict(model)
features_coefs, model["regr"].intercept_

R2: 0.05332478360466941
RMSE: 11.119417413510135
MAE: 8.523911685334067
MAPE: 0.1430179692644062
Valid words: 188
Valuable words: 1


({'chorus': 0.42599177927881754}, 69.25437811083174)